In [0]:
%%sh


PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF  
select * from market. limit 100;

EOF



In [0]:

res = spark.read.csv("s3://prod.appannie.aso.paid.product/oss/DAY/2019-12-01/TS/1/").limit(1)
print res.collect()



In [0]:
%%sh


PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF  

select count(distinct keyword_id) from aso.aso_keyword_trafficshare_fact_v1 where country_code='US' and date='2018-12-01' and device_code='android-all';

EOF



In [0]:

# device_id|country_code|keyword_idsearch_volume|

# df1=spark.read.parquet("s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2020-01-05/platform=1/").filter("country_code='US' and search_volume<20").select("search_volume","device_id").distinct().limit(100).show()
df1=spark.read.parquet("s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2020-01-05/platform=1/").filter("search_volume<20").select("country_code").distinct().collect()
print (len(df1))

In [0]:
%%sh

aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2020-01-05/ --recursive --human-readable --summarize

In [0]:
%%sh

PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF  

select * from limit 10;

EOF


In [0]:

import datetime
from aaintdatapipeline.application.app_qa.common.utils import is_in_current_month, datetime_to_string, \
    get_random_day_in_last_month
from aaintdatapipeline.core.loader import es
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.core.utils.spark import canned_spark, stop
urn = Urn(
    namespace='app-qa.db-check.v1',
    owner='app_qa'
)
common_config = {
    "database": "aso_sov"
}
es_conn = es.connection(urn, common_config)
print es_conn
query_body ={"query": {"bool": {
                      "must": [{"match": {"device_code": "ios-phone"}},
                                 {"match": {"date": "2019-11-01"}}]}},
            "sort": [{"date": {"order": "desc", "mode": "max"}}]}
result = es_conn.search(index, body=query_body)
print result

In [0]:

import datetime
from aaintdatapipeline.application.app_qa.common.utils import is_in_current_month, datetime_to_string, \
    get_random_day_in_last_month
from aaintdatapipeline.core.loader import es
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.core.utils.spark import canned_spark, stop
urn = Urn(
    namespace="app-qa.db-check.v1",
)
common_config = {
    "database": "app_int"
}
es_conn = es.connection(urn, common_config)
sdk_history_indexes = es_conn.indices.get("int_sdk_category_ranking*").keys()
# print sdk_history_indexes
query_body ={
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "country_code": "US"
          }
        },
        {
          "term": {
            "sdk_id":60100000000272
          }
        }
      ]
    }
  }
}
result = es_conn.search(sdk_history_indexes, body=query_body)
print result

In [0]:
%%sh


PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF  

select * from aso.aso_search_visibility_fact_v1 limit 10;

EOF%%sh


PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF  

select * from aso.aso_search_visibility_fact_v1 limit 10;

EOF






















In [0]:

# CITUS Connection
import psycopg2
from conf import settings

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result

def citus_settings(schema):
    template = "CITUS_" + schema.upper() + "_CITUS_DB_{property}"
    host, port = getattr(settings, template.format(property='HOSTS'))[0]
    return (
        "dbname='{db}' user='{user}' password='{password}' "
        "host='{host}' port='{port}'".format(
            db=getattr(settings, template.format(property='NAME')),
            user=getattr(settings, template.format(property='ACCESS_ID')),
            host=host,
            password=getattr(settings, template.format(property='SECRET_KEY')),
            port=port
        )
    )

 
def get_sql(device_code, country_code, table_name):
        sql = '''
             select * from {table_name} where date = '{date}'
                     and device_code = '{device_code}' and country_code = '{country_code}' limit 10;
             '''.format(table_name=table_name, device_code=device_code, date="2019-12-02", country_code=country_code)
        result = query(citus_settings("aso"), sql)
        return result


res = get_sql("ios-phone", "US", "aso.aso_search_visibility_fact_v1")
print res

In [0]:

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.loader import es

# ES Connection
urn = Urn(
    namespace='app-qa.db-check.v1',
    owner='app_qa'
)
query_body = {
                    "query": {"bool": {
                        "must": [{"match": {"device_code": "ios-phone"}},
                                 {"match": {"date": "2019-12-01"}}]}},
                    "sort": [{"date": {"order": "desc", "mode": "max"}}]}
def get_aso_sov_db(country_code, query_body):
        common_config = {
            "database": "aso_sov"
        }
        es_conn = es.connection(urn, common_config)
        res = es_conn.search(
            index="aso-kpi-sov_search_ads_keyword_fact_v1_daily_{}_*".format(country_code.lower()),
            body=query_body)
        return res
res = get_aso_sov_db("US", query_body)
for _v in  res['hits']['hits']:
    print _v['_source']



def get_aso_metrics_date(device_id, country_code):
        common_config = {
            "database": "aso"
        }
        es_conn = es.connection(urn, common_config)
        aso_indexes = es_conn.indices.get(
            "aso-kpi-svi-{}-2019".format(country_code)).keys()
        query_body = {
            "query": {
                "constant_score": {
                    "filter": {
                        "bool": {
                            "must": [
                                {"term": {"day": "2019-11-01"}},
                                {"term": {"device": device_id}}
                            ]
                        }
                    }
                }
            },
            "sort": [
                {"day": {"order": "desc", "mode": "max"}}
            ]
        }
        result = es_conn.search(aso_indexes, body=query_body)
        return result

# res = get_aso_metrics_date(2002, "us")
# print res



In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h pg-n-slave-4edc958bda550bbc.elb.us-east-1.amazonaws.com  -U app_bdp_usage_qa -d aa_amazon -p 5433 << EOF  
select * from category limit 10;

EOF

In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
\d dna_franchise
select * from dna_franchise where franchise_id in(105,108);
select * from dna_franchise_app_mapping  where franchise_id in(105,488,108);
EOF

In [0]:


import datetime
import psycopg2
from contextlib import closing
dbinfo = {
    'NAME': 'dailyest',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt
daily = """SELECT item_id, kpi, app_type_id, est_day, SUM(estimate)::BIGINT as estimate
FROM plproxy.execute_select_nestloop($proxy$
SELECT item_id, kpi, app_type_id, date_trunc('week', (date::date + 1)::date)::date - 1 AS est_day, SUM(estimate)::BIGINT AS estimate
    FROM (
SELECT store_id as item_id, kpi, app_type_id, date, estimate
FROM ms.market_size_daily_estimate_2000
WHERE category_id = 6012
      AND app_type_id in (1, 2)
      AND purchase_type_id = 10
      AND date BETWEEN '2016-12-20'::date and '2016-12-31'
GROUP BY store_id, kpi, app_type_id, date, estimate
        ) tbl
    GROUP BY item_id, kpi, app_type_id, date_trunc('week', (date::date + 1)::date)::date - 1
$proxy$) f (item_id INT, kpi SMALLINT, app_type_id SMALLINT, est_day DATE, estimate BIGINT)
GROUP BY item_id, kpi, app_type_id, est_day
"""
rows, _ = select(daily, None)
print rows

In [0]:


import datetime
import psycopg2
from contextlib import closing
dbinfo = {
    'NAME': 'dailyest',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt


sql = """select * from plproxy.execute_select($$SELECT store_id as item_id, kpi, app_type_id, date, estimate, device_id
FROM ms.market_size_daily_estimate_1000
WHERE category_id = 1
      AND store_id=143441
      AND kpi=1
      AND app_type_id in (2)
      AND purchase_type_id=12
      AND date BETWEEN '2019-12-01' and '2019-12-01';$$)t(item_id INT, kpi SMALLINT, app_type_id SMALLINT, est_day DATE, estimate BIGINT, device_code SMALLINT)"""
rows, _ = select(sql, None)
print len(rows)
print rows




In [0]:
%%sh
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2013-01-01/android/market_size/



In [0]:

res = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-01/android/market_size/10/",sep='\t').filter("_c7='802' and _c0='10' and _c5='0' and _c6='12'")
print res.collect()
res = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-01/ios/market_size/10/",sep='\t').filter("_c7='802' and _c0='10' and _c5='0' and _c6='12'")
print res.collect()
# 802 legacy ios: 7002, android: 39

In [0]:


import datetime
import psycopg2
from contextlib import closing
dbinfo = {
    'NAME': 'dailyest',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt


sql = """select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from plproxy.execute_select_nestloop($proxy$ select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from ms.market_size_daily_estimate_1000 where date = '2019-12-01' and category_id in (7002) and store_id = 10 and app_type_id = 0 and purchase_type_id = 12 and kpi = 1 $proxy$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);"""
rows, _ = select(sql, None)
print len(rows)
print rows




In [0]:

res = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-01/ios/market_size/143441/",sep='\t').filter("_c2=3").show()
#     if i["_c7"] =='36' and i["_c3"]=='ipad' and i["_c4"]=='downloads':
#       print(i)

# _c0         _c1     _c2      _c3        _c4      _c5              _c6                 _c7               _c8
# store_id    date             device     kpi      app_type_id      purchase_type_id    category_id       download&revenue

In [0]:
%%sh
# \d store.store_app_rank_fact_v1
# select * from store.store_app_rank_fact_v1 where country_code='US' and date='2020-01-07' and device_code='ios-phone' limit 100;
# select count(distinct device_code) from store.store_app_rank_fact_v1 where date='2020-01-07' GROUP BY country_code limit 100;


PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
select device_code,count(distinct country_code) from store.store_app_rank_fact_v1 where date='2020-01-07' GROUP BY device_code limit 100;
EOF

In [0]:



# test sdk term

import datetime
from aaintdatapipeline.application.app_qa.common.utils import is_in_current_month, datetime_to_string, \
    get_random_day_in_last_month
from aaintdatapipeline.core.loader import es
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.core.utils.spark import canned_spark, stop
urn = Urn(
    namespace="app-qa.db-check.v1",
)
common_config = {
    "database": "app_int"
}
es_conn = es.connection(urn, common_config)
sdk_history_indexes = es_conn.indices.get("int_sdk_category_ranking*").keys()
# print sdk_history_indexes
query_body ={
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "country_code": "US"
          }
        },
        {
          "term": {
            "sdk_id":60100000000272
          }
        }
      ]
    }
  }
}
result = es_conn.search(sdk_history_indexes, body=query_body)
print result

In [0]:
%%sh

# led store_market_size_latest_date_v1
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
SELECT * FROM store.store_market_size_fact_v1;
EOF























In [0]:
%%sh



PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
SELECT  sum(est_market_size_download) AS est_market_size_download, sum(est_market_size_revenue) as est_market_size_revenue FROM store.store_market_size_fact_v1 WHERE app_price_type_id not in (0) AND purchase_type_id=10 AND device_code in ('ios-all','android-all') AND country_code ='WW' AND date between '2020-02-02' AND '2020-02-08' and category_id  in (100000,400000) ORDER BY est_market_size_download desc;
EOF
















In [0]:
%%sh

# led store_market_size_latest_date_v1
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
SELECT sum(est_market_size_download),sum(est_market_size_revenue) FROM store.store_market_size_fact_v1 where date between '2020-02-16' and '2020-02-22' and category_id in (400000,100000) and device_code in('ios-all','android-all') and country_code='WW' and app_price_type_id=0 and purchase_type_id=10;
EOF

In [0]:
%%sh


PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
select* from usage.usage_genre_cross_fact_v1 limit 100;
EOF